# **Take home assignment**

*Giedrius* Mirklys, 2041025

Computational Linguistics

First, let's import all required packages for this assignment.

In [ ]:
import pandas as pd
import numpy as np
from collections import defaultdict

#!bash # uncomment this line if you want to install the old20 package in the notebook
"""git clone https://github.com/stephantul/old20.git
git clone https://github.com/stephantul/old20.git
exit
""" # this is what you need to write in terminal

!pip install ./old20
!pip install jellyfish

from old20 import old20, old_n 


Processing ./old20
  Created wheel for old20: filename=old20-2.1.0-cp37-none-any.whl size=5473 sha256=674baeae65b6f02e239527ed4003ce4410bd45286f657b8b53761d9c2857ad31
  Stored in directory: /tmp/pip-ephem-wheel-cache-fo3zj9n6/wheels/19/ab/95/1e707b7488437f542a9eaf95d85251151fd5042649abbd239d
Successfully built old20
  Found existing installation: old20 2.1.0
    Uninstalling old20-2.1.0:
      Successfully uninstalled old20-2.1.0


## **Task 1.**

In [ ]:
subtlex_us = pd.read_csv("/content/SUBTLEXus74286wordstextversion.txt", delimiter='\t')
print(subtlex_us.head(10)) # prints first 10 entries of subtlex_us

  Word  FREQcount  CDcount  FREQlow  Cdlow   SUBTLWF  Lg10WF  SUBTLCD  Lg10CD
0  the    1501908     8388  1339811   8388  29449.18  6.1766   100.00  3.9237
1   to    1156570     8383  1138435   8380  22677.84  6.0632    99.94  3.9235
2    a    1041179     8382   976941   8380  20415.27  6.0175    99.93  3.9234
3  you    2134713     8381  1595028   8376  41857.12  6.3293    99.92  3.9233
4  and     682780     8379   515365   8374  13387.84  5.8343    99.89  3.9232
5   it     963712     8377   685089   8370  18896.31  5.9839    99.87  3.9231
6    s    1057301     8377  1052788   8373  20731.39  6.0242    99.87  3.9231
7   of     590439     8375   573021   8372  11577.24  5.7712    99.85  3.9230
8  for     351650     8374   332686   8370   6895.10  5.5461    99.83  3.9230
9    I    2038529     8372     5147    350  39971.16  6.3093    99.81  3.9229


## **Task 2.**

Tokens are all words in a text. However, types are distinct words in a text. So to find all the tokens I created an additional list which stores all word occurances and add the list to the tokens list.

In [ ]:
tokens = []
types = list(subtlex_us["Word"])
for w, freq in zip(list(subtlex_us["Word"]), list(subtlex_us["FREQcount"])):
  wordL = [w]*(int(freq*0.08)) # due to lack of computational power I reduce the token list by 92%. You may uncomment the line below to use all possible tokens and do not forget to delete/comment this then
  #wordL = [w]*freq
  tokens += wordL
tokens = np.array(tokens)
types = np.array(types)

## **Task 3.**

In [ ]:
class LM:

  def __init__(self, tStr=[], ngram_size=0, k=0.01):
    self.tStr = tStr
    self.ngram_size = ngram_size
    self.k = k
    self.transition_matrix()
  
  def transition_matrix(self):
    self.transM = defaultdict(dict) # creating an empty nested dictionary
    self.ngrams = self.create_ngrams() # returns ngram tuples. See the function description for more details in the function itself
    for ngram in self.ngrams:
      history, target = ngram # splits the tuple into two variables
      try:
        self.transM[history][target] += 1 # tries to increase a subdictionary value by 1
      except:
        self.transM[history][target] = 1 # if unsuccessfully, assigns 1.

    for key, trans in self.transM.items():
      # sums up all subdictionary values with smoothing. We multiply the smoothing constant k by 28 because that is how many letters in English alpabet are
      transition_sum = np.sum([count+self.k*28 for count in trans.values()])
      for letter in trans:
        # creates the transition probability of the main dict key and the subdict key. If we don't add the smoothing value, all the probabilities will not sum up to 1
        self.transM[key][letter] = (self.transM[key][letter]+self.k*28)/transition_sum 
  
  def create_ngrams(self): # creates a huge list of ngrams. See create_single_ngram() for the description of how the ngram looks like
    ngrams = []

    for word in self.tStr:
      word = list(word)
      bow = ['#bow#'] if self.ngram_size == 1 else ['#bow#'] * (self.ngram_size-1)
      word = bow+word+['#eow#']
      ngrams.extend(self.create_single_ngram(word))
    return ngrams
  
  def create_single_ngram(self, word):
    """
    This function creates all possible self.ngram_size characters transitions of a single word with its boundaries. 
    So, it creates an n-grams and splits into the past characters and the following character.
    EXAMPLE.
    If we have a word "CAT", and ngram_size = 3. The passed 'word' into this function would look like [#bow#, #bow#, C, A, T, #eow#], #bow# and #eow# mark the word's boundaries.
    The returned list would looke like [((#bow#, #bow#), C), ((#bow#, C), A), ((C, A), T), ((A, T), #eow#)]
    """
    ngram = []
    for i in range(len(word)):
      try: 
        w = []
        l = self.ngram_size + i
        w = word[i:l] # sometimes you can violate the limits of the list, therefore everything is in the try block
        w = (tuple(w[:-1]), w[-1]) # creates a tuple of the tuple of the past self.ngrams-1 characters and the following character.
        assert len(w[0]) == (self.ngram_size-1) # for some reason, the past characters tuple may not be self.ngrams-1 characters long, since I slice the 'word' list until its last value
        ngram.append(w)
      except:
        pass # if it is impossible to append the value, it shall remain ignored. 
    return ngram

  def get_ngram_probability(self, history, target): # calculates the ngrams smoothed transition probability 
    # the code below takes into account that there might not be certain keys in transM
    prob = 0.0
    try:
      prob = self.transM[history][target] 
    except KeyError:
      prob = self.k/28*self.k
        
    return prob 

  def perplexity(self, word): # calculates the perplexity of the given word
    """
    Perplexity is calculated by this formula: find it please in the 40th slide from class 4.
    1. First we take the transition probabilities of the word from the corpus
    2. Take the log values of the probabilities. It must not be log2. It is a choice.
    3. Takes the negative average of all entropy components calculated in step 2.
    4. Takes the second power of the average.

    """
    probs = []
    bow = ['#bow#'] * (self.ngram_size-1)
    word = self.create_single_ngram(bow+list(word)+['#eow#']) # creates an ngram tuple list from a given word
    for ngram in word:
      probs.append(self.get_ngram_probability(ngram[0], ngram[1])) # step 1
                    
    entropy = np.log2(probs) # step 2

    assert all(entropy <= 0) # we cant have all values of 0

    avg_entropy = -1 * (sum(entropy) / len(entropy)) # step 3
    
    return pow(2.0, avg_entropy) # step 4

  def generate(self, limit, generation='random'):
    i = 0
    r = 1 if self.ngram_size == 1 else self.ngram_size - 1
    word = ['#bow#']*r
    current = word[-(self.ngram_size-1):] # It needs something to start. 
    while i < limit:
        letters = []
        probabilities = []
        continuations = self.transM[tuple(current)] # otherwise it could not take a subdictionary to generate the next character
        for l, v in continuations.items():
            letters.append(l)
            probabilities.append(v)

        # it can generate a character in three different ways: random from the probability distribution, with the highest probability, and with the lowest probability.    
        if generation == 'random':
          new = np.random.choice(letters, size=1, p=probabilities)[0]
        elif generation == 'highest':
          new = letters[np.argmax(probabilities)]
        elif generation == 'lowest':
          new = letters[np.argmin(probabilities)] 
        
        # below, it checks if it encounters the end character, or it generates words until the limit of characters.

        if new != '#eow#': 
            word.append(new) 
        else: 
            return ''.join(word[self.ngram_size-1:])

        current = word[-(self.ngram_size-1):]   
        i += 1

    return ''.join(word[self.ngram_size-1:])


In [ ]:
ngram_model = LM(tStr=tokens, ngram_size=5)


In [ ]:
[ngram_model.generate(50) for _ in range(50)] # you can test the pentagram token model. It generates quite nice words. Tap tap on myself.

['go',
 'won',
 'to',
 'a',
 'end',
 'place',
 'stop',
 'the',
 'looks',
 'so',
 'your',
 'to',
 'way',
 'fine',
 'cent',
 'be',
 'the',
 's',
 'oping',
 'floor',
 'you',
 'over',
 'big',
 'asylum',
 'Sooking',
 'they',
 'he',
 'better',
 'the',
 'included',
 't',
 'law',
 'our',
 'who',
 'you',
 'morning',
 'a',
 'and',
 'because',
 'he',
 'going',
 'Saturiness',
 'you',
 'though',
 'it',
 'close',
 'arms',
 'bodies',
 'shit',
 'everywhere']

## **Task 4**

In [ ]:
trigram_token = LM(tokens, 3)
trigram_type = LM(types, 3)
tetragram_token = LM(tokens, 4)
tetragram_type = LM(types, 4)

In [ ]:
def lowest_perplexity(lm=LM("", 0), strs=types, length=0): # calculates lowest perplexity of a certain length words
  p = []
  w2 = []
  for w in strs:
    if len(w) == length:
      p.append(lm.perplexity(w))
      w2.append(w)
  return (w2[p.index(min(p))], min(p)) # p.index(min(p)) outputs the index where the min(p) is.

In [ ]:
"""
This cell and the cell below helps to create a nice dataframe of all 4 language models.
"""

def create_dict_lowest_perplexity(lm=LM("", 0), strs=types, length=0, input=""):
  d = {}
  lp = lowest_perplexity(lm, strs, length)
  d["Context"] = "{}-gram".format(lm.ngram_size)
  d["Input"] = inputcustom functions
  d["Word"] = lp[0]
  d["Perplexity"] = lp[1]

  return d


In [ ]:
dd = defaultdict(list)
md = []
for l in [3, 8, 13]:
  md.append(create_dict_lowest_perplexity(lm=trigram_token, length=l, input='token'))
  md.append(create_dict_lowest_perplexity(lm=trigram_type, length=l, input='type'))
  md.append(create_dict_lowest_perplexity(lm=tetragram_token, length=l, input='token'))
  md.append(create_dict_lowest_perplexity(lm=tetragram_type, length=l, input='type'))

for d in md:
    for key, value in d.items():
        dd[key].append(value)

pd.DataFrame(dd)

,Context,Input,Word,Perplexity
0,3-gram,token,you,3.130281
1,3-gram,type,ing,4.177166
2,4-gram,token,you,2.807220
3,4-gram,type,man,7.101688
4,3-gram,token,anything,4.682576
5,3-gram,type,mentions,5.299059
6,4-gram,token,anything,3.227310
7,4-gram,type,stations,4.005235
8,3-gram,token,motherfucking,6.241695
9,3-gram,type,fractionating,5.699148


## **Task 5**

The cells below generate the likeliest strings of 4 different language models and creates a nice dataframe storing all asked information.

In [ ]:
def create_dict_likeliest_string(lm=LM("", 0), input="", size=6):
  d = {}
  w = lm.generate(size, "highest")
  d["Context"] = "{}-gram".format(lm.ngram_size)
  d["Input"] = input
  d["Word"] = w

  return d


In [ ]:
dd = defaultdict(list)
md = []
md.append(create_dict_likeliest_string(lm=trigram_token, input='token'))
md.append(create_dict_likeliest_string(lm=trigram_type, input='type'))
md.append(create_dict_likeliest_string(lm=tetragram_token, input='token'))
md.append(create_dict_likeliest_string(lm=tetragram_type, input='type'))

for d in md:
    for key, value in d.items():
        dd[key].append(value)

pd.DataFrame(dd)

,Context,Input,Word
0,3-gram,token,the
1,3-gram,type,st
2,4-gram,token,the
3,4-gram,type,stant


## **Task 6**

In [ ]:
"""
Each language model generates 10 different strings. 
Also, the levenshtein distances of 20 closest words are found for each string group.
"""

token_trigram = []
type_trigram = []
token_tetragram = []
type_tetragram = []

while len(token_trigram) <= 10 or len(type_trigram) <= 10 or len(token_tetragram) <= 10 or len(type_tetragram) <= 10:

  w = trigram_token.generate(50)
  if w not in token_trigram and len(token_trigram) <= 10:
    token_trigram.append(w)

  w = trigram_type.generate(50)
  if w not in type_trigram and len(type_trigram) <= 10:
    type_trigram.append(w)

  w = tetragram_token.generate(50)
  if w not in token_tetragram and len(token_tetragram) <= 10:
    token_tetragram.append(w)

  w = tetragram_type.generate(50)
  if w not in type_tetragram and len(type_tetragram) <= 10:
    type_tetragram.append(w)

token_trigram_lev = old_n(token_trigram, types, n=20)
type_trigram_lev = old_n(type_trigram, types, n=20)
token_tetragram_lev = old_n(token_tetragram, types, n=20)
type_tetragram_lev = old_n(type_tetragram, types, n=20)



In [ ]:
def create_dict_old20(g_strings=[], old20_values=[], input="", ngram_s=0):
  d = {}
  d["N-gram size"] = "{}".format(ngram_s) # let it be
  d["Input"] = input
  d["Average OLD20 of the generated strings"] = np.average(np.array(old20_values))
  d["Average length of the generated strings"] = np.average(np.array([len(x) for x in g_strings]))

  return d

In [ ]:
dd = defaultdict(list)
md = []
md.append(create_dict_old20(token_trigram, token_trigram_lev, "token", 3))
md.append(create_dict_old20(type_trigram, type_trigram_lev, "type", 3))
md.append(create_dict_old20(token_tetragram, token_tetragram_lev, "token", 4))
md.append(create_dict_old20(type_tetragram, type_tetragram_lev, "type", 4))

for d in md:
    for key, value in d.items():
        dd[key].append(value)
pd.DataFrame(dd)

,Ngram size,Input,Average OLD20 of the generated strings,Average length of the generated strings
0,3,token,30.818182,3.636364
1,3,type,100.909091,10.090909
2,4,token,22.727273,2.909091
3,4,type,57.000000,7.181818




* Which language model generates new strings with denser orthographic neighbourhoods (lower OLD20 values)?

> We can see from the data frame that it is the 4-gram token model.


* How does the length of generated strings factor in? 

> Briefly, the shorter word, the fewer changes you need to make to get a wanted string. And the avrage OLD20 shows the average levenshtein distance (the min number of a single character changes) between the generated words and 20 closest neighbors in the corpus.

* What does that tell us with respect to the different language models and their ability to capture English orthotactics?

> Other models generate strings which are less likely to be present in the English language or to follow "the rules" of how English words are formed. We can see that type based models generate quite long and non-English strings. Moreover, in the English language, mojority of the most frequent words are short, so long words would less likely conform to English orthotactics. 



## **Task 7**

In [ ]:
def perplexity_calc(w):
  """
  outputs the dictionary of perplexity values of all 4 languge models
  """
  d = defaultdict(dict)
  d1, d2, d3, d4 = {}, {}, {}, {}
  for i in w:
    d1[i] = trigram_token.perplexity(i)
    d2[i] = trigram_type.perplexity(i)
    d3[i] = tetragram_token.perplexity(i)
    d4[i] = tetragram_type.perplexity(i)
  d['3 token'] = d1
  d['3 type'] = d2
  d['4 token'] = d3
  d['4 type'] = d4
  return d


In [ ]:
def create_dict_perplexity_langugage(lang='', perplexity_scores={}, words=[], rank=''):
  assert rank == 'highest' or rank == 'lowest' # rank here means what kind of perplexity values one wants to generate: only the highest perplexity values or only the lowest
  d = []
  for model in perplexity_scores:
    dd = {}
    dd['n-gram size'] = model.split()[0]
    dd['input dataset'] = words
    dd['concept'] = model.split()[1]
    dd['language'] = lang
    if rank == 'lowest':
      w = min(perplexity_scores[model])
    elif rank == 'highest':
      w = max(perplexity_scores[model])
    dd['word'] = w
    dd['perplexity'] = perplexity_scores[model][w]
    d.append(dd)
  return d

In [ ]:
def average_perplexity_language(lang='', perplexity_scores={}, words=[]):
  d = []
  for model in perplexity_scores:
    dd = {}
    dd['n-gram size'] = model.split()[0]
    dd['input dataset'] = words
    dd['concept'] = model.split()[1]
    dd['language'] = lang
    dd['average perplexity'] = np.average(np.array(list(perplexity_scores[model].values())))
    d.append(dd)
  return d

In [ ]:
def appendTo_DataFrame(df, x):
  """
  Appends each value of the 'x' variable to the dataframe 'df' and returns the dataframe
  """
  for entry in x:
    df = df.append(entry, ignore_index=True)
  return df

In [ ]:
no 

eng = perplexity_calc(english_words)
basq = perplexity_calc(basque_words)
czech = perplexity_calc(czech_words)
dutch = perplexity_calc(dutch_words)
fin = perplexity_calc(finnish_words)
ital = perplexity_calc(italian_words)

highest_perplexity = pd.DataFrame()
lowest_perplexity = pd.DataFrame()
average_perplexity = pd.DataFrame()

h = [create_dict_perplexity_langugage('english', eng, english_words, 'highest'),
     create_dict_perplexity_langugage('basque', basq, basque_words, 'highest'),
     create_dict_perplexity_langugage('czech', czech, czech_words, 'highest'),
     create_dict_perplexity_langugage('dutch', dutch, dutch_words, 'highest'),
     create_dict_perplexity_langugage('finnish', fin, finnish_words, 'highest'),
     create_dict_perplexity_langugage('italian', ital, italian_words, 'highest')]
highest_perplexity = appendTo_DataFrame(highest_perplexity, h)

l = [create_dict_perplexity_langugage('english', eng, english_words, 'lowest'),
     create_dict_perplexity_langugage('basque', basq, basque_words, 'lowest'),
     create_dict_perplexity_langugage('czech', czech, czech_words, 'lowest'),
     create_dict_perplexity_langugage('dutch', dutch, dutch_words, 'lowest'),
     create_dict_perplexity_langugage('finnish', fin, finnish_words, 'lowest'),
     create_dict_perplexity_langugage('italian', ital, italian_words, 'lowest')]
lowest_perplexity = appendTo_DataFrame(lowest_perplexity, l)

a = [average_perplexity_language('english', eng, english_words),
     average_perplexity_language('basque', basq, basque_words),
     average_perplexity_language('czech', czech, czech_words),
     average_perplexity_language('dutch', dutch, dutch_words),
     average_perplexity_language('finnish', fin, finnish_words),
     average_perplexity_language('italian', ital, italian_words)]
average_perplexity = appendTo_DataFrame(average_perplexity, a)

# please take off the quotation marks from the docstring below to print all the dataframes

"""print("highest", highest_perplexity, sep='\n')
print('lowest', lowest_perplexity, sep='\n')
print('average', average_perplexity, sep='\n')"""

average_perplexity


,n-gram size,input dataset,concept,language,average perplexity
0,3,"[mouth, dream, sun, apple, bridge, mirror, sky...",token,english,17.424964
1,3,"[mouth, dream, sun, apple, bridge, mirror, sky...",type,english,13.613626
2,4,"[mouth, dream, sun, apple, bridge, mirror, sky...",token,english,11.055962
3,4,"[mouth, dream, sun, apple, bridge, mirror, sky...",type,english,10.688955
4,3,"[aho, amets, eguzki, sagar, zubi, mirail, zeru...",token,basque,104.518404
5,3,"[aho, amets, eguzki, sagar, zubi, mirail, zeru...",type,basque,38.089716
6,4,"[aho, amets, eguzki, sagar, zubi, mirail, zeru...",token,basque,68.414639
7,4,"[aho, amets, eguzki, sagar, zubi, mirail, zeru...",type,basque,33.332370
8,3,"[pusa, sen, slunce, jablko, most, zrcadlo, neb...",token,czech,71.535660
9,3,"[pusa, sen, slunce, jablko, most, zrcadlo, neb...",type,czech,31.156011




> What can you say about relations between the target languages and English based on how perplex the language models are when fed with translations of these 10 core concepts? 

To answer the question, I shall refer to the average perplexity table as it represents the scores in the best way. Obsiously, the English language's perplexity scores are the lowest. Therefore, they can be base values for comparing perplexities of different languages. As it could be foreseen beforehand, the Dutch language is the most similar to English since the perplexity values are the closest. Moreover, the Basque language is very distinct from English language. Fascinatingly, Italian orthotactics are no so different from English ones'. Though, you can find very similar words to English in Italian indeed. 



### **Some notes for the grader**

Python is a pretty self-explanatory language, and I do believe I should not explain each line. For example, when I add something to a list or create a simple dictionary. 

Also, some lines are indented with 2 spaces, some are with 4. Please do not be bothered by it. If for some reason it does not work, you are free to edit it. But as far as I know, we are allowed to use either 2-space or 4-space indentation. Perhaps it depends on the version. 

Finally, I spotted that I get an inappropriate word as an output. So I am not the one who you should blame here. Blame the data :)